# Functions to create the table

In [ ]:
function meanmin(A::Alloc,T::Taxes,M::Para,N::Hyper)
    wages = [wage(i,j,A,T,M,N) for i in 1:N.I, j in 1:N.J]
    
    MmJ = zeros(N.J)
    for j in 1:N.J 
        mini = 100. # minimum(wages[:,j]), better to find mini among n>0
        num = 0.
        den = 0.
        for i in 1:N.I
            if A.n[i,j]>0
                num += A.n[i,j] * wages[i,j]
                den += A.n[i,j]
                mini =  min(mini , wages[i,j] )
            end
        end
        MmJ[j] = num/den / mini
    end
    
    MmI = zeros(N.I)
    for i in 1:N.I 
        mini = 100. #minimum(wages[i,:])
        num = 0.
        den = 0.
        for j in 1:N.J
            if A.n[i,j]>0
                num += A.n[i,j] * wages[i,j]
                den += A.n[i,j]
                mini =  min(mini , wages[i,j] )
            end
        end
        MmI[i] = num/den / mini
    end
    
    return(MmJ,MmI)
end
    


In [ ]:
function ty_stat(oA::Alloc,oT::Taxes,M::Para,N::Hyper)
    ## define the three vectors
    nn = oA.n
    yy = M.y
    tt = [T_tax(i,j, oA.X, oT,M) * (M.r+M.delta[j]) for i in 1:N.I, j in 1:N.J]

    return( cov(reshape(tt,N.J^2),reshape(yy,N.J^2))/cov(reshape(yy,N.J^2),reshape(yy,N.J^2)) )
    
    #=## compute means
    my = sum(nn .* yy) / sum(nn)
    mt = sum(nn .* tt) / sum(nn)
    
    ## correlation
    num = sum(   nn .* (M.y .- my) .* (tt .- mt)    )
    den1 = sqrt(sum(  nn .* (M.y .- my).^2  ))   
    den2 = sqrt(sum(  nn .* (tt .- mt).^2  ))
    
    return(num/(den1*den2))=#
end

function ty_stat2(oA::Alloc,oT::Taxes,M::Para,N::Hyper)
    ## define the three vectors
    nn = oA.n
    yy = M.y
    tt = [T_tax(i,j, oA.X, oT,M) * (M.r+M.delta[j]) for i in 1:N.I, j in 1:N.J]
    wages = [wage(i,j,oA,oT,M,N) for i in 1:N.I, j in 1:N.J]
    tt .-= 3/7 * wages
    
    
    return( cov(reshape(tt,N.J^2),reshape(yy,N.J^2))/cov(reshape(yy,N.J^2),reshape(yy,N.J^2)) )
    
    #=## compute means
    my = sum(nn .* yy) / sum(nn)
    mt = sum(nn .* tt) / sum(nn)
    
    ## correlation
    num = sum(   nn .* (M.y .- my) .* (tt .- mt)    )
    den1 = sqrt(sum(  nn .* (M.y .- my).^2  ))   
    den2 = sqrt(sum(  nn .* (tt .- mt).^2  ))
    
    return(num/(den1*den2))=#
end

We find the measures of vacancies with
\begin{align}
	\theta_{j}=\frac{v_{j}}{\int_{\mathcal{I}}s(e_{ij}^0)u_idi+\int_{\mathcal I\times\mathcal J}\xi s(e_{ij}^{j_0})n_{ij_0}di dj_0}.\label{eq:thetaj_ext}
\end{align}

In [ ]:
function vacancies(A::Alloc,M::Para,N::Hyper)
    v = zeros(N.J)

    for j in 1:N.J
        denj = 0
        for i in 1:N.I
            denj += s(A.e0[i,j],M) * A.u[i]
        end
        for i in 1:N.I, j0 in 1:N.J
            denj += M.xi * s(A.e1[i,j0,j],M) * A.n[i,j0]
        end
        v[j] = A.theta[j] * denj 
    end
    
    return(v)
end